In [2]:
#API
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

#ETL
import pandas as pd
from pandas import json_normalize
import time
import json
import numpy as np
from datetime import datetime, timezone
from pathlib import Path

#DB
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

#Other
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()
api_key = os.getenv('api_key')
db_password = os.getenv('db_password')
db_user = os.getenv('db_user')
db_host = os.getenv('db_host')
db_port = os.getenv('db_port')
db_name = os.getenv('db_name')

In [4]:
headers = {"Authorization": f"Bearer {api_key}"}

In [50]:
championship_map = { 
    #reg season 
    '786da1fa-a9b5-4d17-b40e-416588272db4' : 'Advanced',
    'bd7957b3-1114-435f-af43-22645a508cc0': 'Main',
    '4410aa73-9584-4500-a7ca-1504ed6da707': 'Intermediate',
    'e61be909-1727-4de3-8201-419d89893401':'Entry',
    'f62c4c75-f620-40a2-bef8-0be2cb85d2d3': 'Open10',
    '0ce757ef-9bad-4afa-bbce-58d647824222':'Open9',
    '3cd81a7e-ed93-43ba-be08-2a1ee7ad7237':'Open1-8',
    #playoffs
    'd66d7f6f-3430-4dc3-a0e9-0fb8d9b5e68d':'Advanced',
    '70a6c0b5-c870-4c31-be83-d55a811514af':'Main',
    '36a6a3d1-6024-40c5-ba26-1f33c173eb30':'Intermediate',
    'ff3a5522-8926-4ccc-879b-9a9e1766974d':'Intermediate',
    '3704ee10-0a67-4cf9-8f62-dc6f5cdfc0c1':'Entry',
    'a3354661-a9e0-4375-8387-ece9aae8aadc':'Entry',
    '1e94abfd-f0e8-47cc-b450-95cccaf954e5':'Open10',
    '56b2e3c1-e98d-4196-85df-af8f527ede5b':'Open10',
    '662b2b3b-8769-4806-b207-e92b3b0b2816':'Open9',
    '736302f9-3fb4-46d4-ae5c-5633e842495d':'Open1_8'}

## Call API pour récupérer les données

In [6]:
data_dir = r"D:\Data project\Faceit dashboard\FaceitApi_Dataproject\data"
season = "Season 54"
region = "EU"
stage = "Regular"
start_season_unix = 1751328000

In [7]:
limit = 100
delay = 0.5
max_threads = 5

### List Teams and Team_leaders from championship api call

In [16]:
def fetch_incremental_matches(championship_map, headers,
                              season=season, region=region, stage=stage,
                              data_dir=data_dir, limit=100, delay=0.3):
    base_path = os.path.join(data_dir, season, region, stage)
    os.makedirs(base_path, exist_ok=True)

    # Load existing team_details.json
    team_file = os.path.join(base_path, "team_list.json")
    try:
        with open(team_file, "r") as f:
            team_list = json.load(f)
    except FileNotFoundError:
        team_list = []

    existing_team_ids = {team["team_id"] for team in team_list}

    # Load existing master.json
    master_file = os.path.join(base_path, "master.json")
    try:
        with open(master_file, "r") as f:
            master_matches = json.load(f)
    except FileNotFoundError:
        master_matches = []

    existing_match_ids = {match["match_id"] for match in master_matches}

    parameters = ["upcoming", "past", "all"]

    for champ_id, champ_name in championship_map.items():
        champ_folder = os.path.join(base_path, champ_name)
        os.makedirs(champ_folder, exist_ok=True)
        champ_file = os.path.join(champ_folder, "matches.json")

        # Load existing matches
        try:
            with open(champ_file, "r") as f:
                champ_matches = json.load(f)
        except FileNotFoundError:
            champ_matches = []

        champ_match_ids = {m["match_id"] for m in champ_matches}

        for param in parameters:
            offset = 0
            while True:
                url = f"https://open.faceit.com/data/v4/championships/{champ_id}/matches?region={region}&type={param}&limit={limit}&offset={offset}"
                response = requests.get(url, headers=headers)
                if response.status_code != 200:
                    print(f"Error fetching {champ_name} ({champ_id}) offset {offset} param={param}: {response.status_code}")
                    break

                data = response.json()
                items = data.get("items", [])
                if not items:
                    break

                # Filter new matches
                new_items = [m for m in items if m["match_id"] not in champ_match_ids]
                if new_items:
                    champ_matches.extend(new_items)
                    champ_match_ids.update([m["match_id"] for m in new_items])
                    print(f"Added {len(new_items)} new matches to {champ_file}")

                    # Update master.json
                    for m in new_items:
                        if m["match_id"] not in existing_match_ids:
                            master_matches.append(m)
                            existing_match_ids.add(m["match_id"])

                        # Extract teams and leaders
                        teams = m.get("teams", {})
                        for faction in teams.values():
                            if isinstance(faction, dict):
                                team_id = faction.get("faction_id")
                                if team_id and team_id not in existing_team_ids:
                                    team_info = {
                                        "team_id": team_id,
                                        "name": faction.get("name"),
                                        "avatar": faction.get("avatar"),
                                        "type": faction.get("type"),
                                        "leader_id": faction.get("leader")
                                    }
                                    team_list.append(team_info)
                                    existing_team_ids.add(team_id)

                offset += limit
                time.sleep(delay)

        # Save championship matches
        with open(champ_file, "w") as f:
            json.dump(champ_matches, f, indent=2)

    # Save master.json and team_list.json
    with open(master_file, "w") as f:
        json.dump(master_matches, f, indent=2)

    with open(team_file, "w") as f:
        json.dump(team_list, f, indent=2)

    print(f"Incremental fetch finished. Total matches in master: {len(master_matches)}")
    print(f"Total teams in team_list.json: {len(team_list)}")

    return master_matches, team_list

master_matches, team_list = fetch_incremental_matches(championship_map, headers)

Error fetching Main (bd7957b3-1114-435f-af43-22645a508cc0) offset 600 param=past: 400
Error fetching Main (bd7957b3-1114-435f-af43-22645a508cc0) offset 600 param=all: 400
Error fetching Intermediate (4410aa73-9584-4500-a7ca-1504ed6da707) offset 600 param=past: 400
Error fetching Intermediate (4410aa73-9584-4500-a7ca-1504ed6da707) offset 600 param=all: 400
Error fetching Entry (e61be909-1727-4de3-8201-419d89893401) offset 600 param=past: 400
Error fetching Entry (e61be909-1727-4de3-8201-419d89893401) offset 600 param=all: 400
Error fetching Open10 (f62c4c75-f620-40a2-bef8-0be2cb85d2d3) offset 600 param=past: 400
Error fetching Open10 (f62c4c75-f620-40a2-bef8-0be2cb85d2d3) offset 600 param=all: 400
Error fetching Open9 (0ce757ef-9bad-4afa-bbce-58d647824222) offset 600 param=past: 400
Error fetching Open9 (0ce757ef-9bad-4afa-bbce-58d647824222) offset 600 param=all: 400
Error fetching Open1-8 (3cd81a7e-ed93-43ba-be08-2a1ee7ad7237) offset 600 param=past: 400
Error fetching Open1-8 (3cd81a7e

In [8]:
with open(r"data\Season 54\EU\Regular\master.json", "r", encoding="utf-8") as f:
    master_matches = json.load(f)

with open(r"data\Season 54\EU\Regular\team_list.json", "r", encoding="utf-8") as f:
    team_list = json.load(f)

### List all recent matches from captains and filter those being ESEA related

In [8]:
team_details_file = os.path.join(data_dir, season, region, stage, "team_list.json")
output_file = os.path.join(data_dir, season, region, stage, "all_captain_matches.json")
os.makedirs(os.path.dirname(output_file), exist_ok=True)

def fetch_captain_matches(player_id, existing_matches, from_timestamp, api_key, limit, delay):
    headers = {"Authorization": f"Bearer {api_key}"}
    match_info = existing_matches.copy()
    offset = 0

    while True:
        url = f"https://open.faceit.com/data/v4/players/{player_id}/history?game=cs2&from={from_timestamp}&limit={limit}&offset={offset}"
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print(f"Error for {player_id}: {response.status_code}")
            break

        data = response.json()
        matches = data.get("items", [])

        if not matches:
            break

        new_count = 0
        for match in matches:
            match_id = match["match_id"]
            if match_id not in match_info:
                competition_id = match.get("competition_id")
                started_at = match.get("started_at")
                faction1_team_id = match.get("teams", {}).get("faction1", {}).get("team_id")
                faction2_team_id = match.get("teams", {}).get("faction2", {}).get("team_id")
                winner = match.get("results", {}).get("winner")

                match_info[match_id] = {
                    "competition_id": competition_id,
                    "started_at": started_at,
                    "faction1_team_id": faction1_team_id,
                    "faction2_team_id": faction2_team_id,
                    "winner": winner
                }
                new_count += 1

        print(f"{player_id} offset {offset}: {len(matches)} fetched, {new_count} new matches added")
        offset += limit

        if len(matches) < limit:
            break

        time.sleep(delay)

    return player_id, match_info


def fetch_incremental_captain_matches(captains_ids, api_key, start_season_unix,
                                      filename=output_file, limit=100, delay=0.5, max_threads=5):

    # Load existing data
    if os.path.exists(filename):
        with open(filename, "r") as f:
            all_captain_matches = json.load(f)
        print(f"Loaded existing captain matches from {filename}")
    else:
        all_captain_matches = {}

    # Determine global latest timestamp among all existing captains
    latest_global_timestamp = 0
    for matches in all_captain_matches.values():
        for match in matches.values():
            ts = match.get("started_at", 0)
            if ts > latest_global_timestamp:
                latest_global_timestamp = ts

    print(f"Global latest started_at timestamp: {latest_global_timestamp}")

    # Counter for new matches across all captains
    total_new_matches = 0

    # Prepare tasks
    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = {}
        for player_id in captains_ids:
            existing_matches = all_captain_matches.get(player_id, {})
            if existing_matches:
                from_timestamp = latest_global_timestamp + 1
            else:
                from_timestamp = int(start_season_unix)

            future = executor.submit(
                fetch_captain_matches,
                player_id,
                existing_matches,
                from_timestamp,
                api_key,
                limit,
                delay
            )
            futures[future] = player_id

        # Collect results
        for future in as_completed(futures):
            player_id, match_info = future.result()
            before = len(all_captain_matches.get(player_id, {}))
            after = len(match_info)
            new_for_player = after - before
            total_new_matches += new_for_player

            all_captain_matches[player_id] = match_info
            print(f"{player_id}: {new_for_player} new matches, total now {after}")

    # Save updated JSON
    with open(filename, "w") as f:
        json.dump(all_captain_matches, f, indent=2)

    print(f"✅ All captain matches updated in {filename}")
    print(f"🎯 Total new matches fetched this run: {total_new_matches}")

    return all_captain_matches, total_new_matches


with open(team_details_file, "r") as f:
    teams = json.load(f)

captains_ids = list({team["leader_id"] for team in teams if "leader_id" in team})
print(f"Found {len(captains_ids)} captains to fetch.")

all_captain_matches, total_new_matches = fetch_incremental_captain_matches(
    captains_ids, api_key, start_season_unix, limit=limit, delay=delay, max_threads=max_threads
)


Found 1339 captains to fetch.
Loaded existing captain matches from D:\Data project\Faceit dashboard\FaceitApi_Dataproject\data\Season 54\EU\Regular\all_captain_matches.json
Global latest started_at timestamp: 1756773254
25ccf0f1-2edd-472d-bde9-ed6a5525d2ed offset 0: 8 fetched, 7 new matches added
25ccf0f1-2edd-472d-bde9-ed6a5525d2ed: 7 new matches, total now 200
3deb854e-37e0-4fdd-8560-f0de6ae42c3c offset 0: 7 fetched, 7 new matches added
3deb854e-37e0-4fdd-8560-f0de6ae42c3c: 7 new matches, total now 359
3a915831-0dca-46ec-b9db-b1183ef8a4bf: 0 new matches, total now 230
a12244bf-c5f7-40b2-b70c-254cebbaf16d: 0 new matches, total now 168
61893847-66af-43f3-ad62-d43605ad76cc offset 0: 5 fetched, 5 new matches added
61893847-66af-43f3-ad62-d43605ad76cc: 5 new matches, total now 164
e2c48863-bbe2-4ac2-9f82-d63fa4af8556: 0 new matches, total now 107
Error for : 404
: 0 new matches, total now 0
726f58cd-24d8-4cd2-81a0-5521d627a8e7 offset 0: 7 fetched, 7 new matches added
726f58cd-24d8-4cd2-81

In [41]:
with open(r"data\Season 54\EU\Regular\all_captain_matches.json", "r", encoding="utf-8") as f:
    all_captain_matches = json.load(f)

In [43]:
rows = []
for player_id, matches in all_captain_matches.items():
    for match_id, match_info in matches.items():
        rows.append({
            "match_id": match_id,
            "competition_id": match_info.get("competition_id"),
            "match_day": match_info.get("started_at"),
            "faction1_team_id" : match_info.get("faction1_team_id"),
            "faction2_team_id": match_info.get("faction2_team_id"),
            "winner": match_info.get("winner"),
        })

# Convert to DataFrame
all_captain_matches_df = pd.DataFrame(rows)

# Optional: convert started_at to datetime
all_captain_matches_df["match_day"] = pd.to_datetime(all_captain_matches_df["match_day"], unit="s")

In [51]:
filtered_captain_matches_df = all_captain_matches_df[all_captain_matches_df['competition_id'].isin(championship_map)]
filtered_captain_matches_df = filtered_captain_matches_df.drop_duplicates(subset=['match_id'], keep='first')
filtered_captain_matches_df['winner_id'] = np.where(
    filtered_captain_matches_df['winner'] == 'faction1',
    filtered_captain_matches_df['faction1_team_id'],
    filtered_captain_matches_df['faction2_team_id']
)
filtered_captain_matches_df['loser_id'] = np.where(
    filtered_captain_matches_df['winner'] == 'faction1',
    filtered_captain_matches_df['faction2_team_id'],
    filtered_captain_matches_df['faction1_team_id']
)

filtered_captain_matches_df = filtered_captain_matches_df.drop(columns={'faction1_team_id','faction2_team_id','winner'})
filtered_captain_matches_df = filtered_captain_matches_df.reset_index(drop=True)

In [52]:
filtered_captain_matches_df

,match_id,competition_id,match_day,winner_id,loser_id
0,1-f197f4c1-6c52-46f6-b66b-edd2f5830813,e61be909-1727-4de3-8201-419d89893401,2025-08-18 18:07:19,83756527-b019-449f-9660-adc29f01416f,cfdfa687-0075-4f23-8129-28eb01b37a2a
1,1-8d4a43ae-90a0-48bb-a575-c089699bca24,e61be909-1727-4de3-8201-419d89893401,2025-08-13 18:41:24,97f7d131-9ffe-4dde-a8e5-e2dcee2113ea,cfdfa687-0075-4f23-8129-28eb01b37a2a
2,1-89b9853e-781a-4ca6-bb34-5ee736e20a50,e61be909-1727-4de3-8201-419d89893401,2025-08-11 18:39:34,cfdfa687-0075-4f23-8129-28eb01b37a2a,db190109-606c-4845-a773-574dde647db3
3,1-d606f510-8000-4f18-a438-5712554f4d49,e61be909-1727-4de3-8201-419d89893401,2025-08-10 18:36:18,aeff8fa0-734c-45bb-8b9b-f18847d630f6,cfdfa687-0075-4f23-8129-28eb01b37a2a
4,1-5517b202-c958-4244-8688-d123d637e178,e61be909-1727-4de3-8201-419d89893401,2025-08-04 18:39:52,cfdfa687-0075-4f23-8129-28eb01b37a2a,dc0203c1-9604-4ad0-bdea-644b117609b3
...,...,...,...,...,...
9409,1-7a5d0f6e-e6ef-41bb-8504-137f483ae33f,d66d7f6f-3430-4dc3-a0e9-0fb8d9b5e68d,2025-09-01 20:16:42,0c06fb32-9c32-4a48-8967-943baf52ab6a,bye
9410,1-40b28985-c35e-4124-9f2a-30a543d730dd,4410aa73-9584-4500-a7ca-1504ed6da707,2025-08-12 18:14:10,b3e21e6e-7e76-4cba-b3af-b65b91833d6c,df862c73-6fc9-4f84-8160-22dd06293fd3
9411,1-b202e7b4-9f28-433f-8043-bcdff80d4048,e61be909-1727-4de3-8201-419d89893401,2025-07-23 18:39:19,4269aee6-d662-4d1b-80c9-495904bd65ed,7895ce74-408e-421d-b83a-05337ac700d4
9412,1-d9206de0-f03f-456e-9c56-f466e7de9f46,e61be909-1727-4de3-8201-419d89893401,2025-07-23 18:39:51,bdc870bc-c386-4488-9084-6df1ff584794,270c07be-ae43-4628-bc95-e3776f167dee


In [53]:
# Define the path to your stats file
stats_file = Path(r"data\Season 54\EU\Regular\detailed_match_stats.json")

# Load existing stats or start empty
if stats_file.exists():
    with stats_file.open("r", encoding="utf-8") as f:
        all_stats = json.load(f)
else:
    all_stats = {}

# Keep track of already fetched match IDs
existing_match_ids = set(all_stats.keys())

In [54]:
new_match_ids = filtered_captain_matches_df[
    ~filtered_captain_matches_df["match_id"].isin(existing_match_ids)
]["match_id"].tolist()

print(f"{len(new_match_ids)} new matches to fetch stats for.")


2194 new matches to fetch stats for.


### Pull detailed stats of all ESEA matches found

In [14]:
def fetch_single_match(match_id):
    url = f"https://open.faceit.com/data/v4/matches/{match_id}/stats"
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 404:
            return match_id, None
        response.raise_for_status()
        time.sleep(delay)
        return match_id, response.json()
    except requests.exceptions.RequestException:
        return match_id, None

# --------------------------
# 6. Threaded fetching
# --------------------------
if new_match_ids:
    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = {executor.submit(fetch_single_match, mid): mid for mid in new_match_ids}
        for i, future in enumerate(as_completed(futures), start=1):
            match_id, data = future.result()
            if data is not None:
                all_stats[match_id] = data
                print(f"[{i}/{len(new_match_ids)}] {match_id} fetched ✅")
            else:
                print(f"[{i}/{len(new_match_ids)}] {match_id} has no stats ❌")
else:
    print("No new matches to fetch.")

# --------------------------
# 7. Save updated stats
# --------------------------
with stats_file.open("w", encoding="utf-8") as f:
    json.dump(all_stats, f, indent=2)

print(f"\n✅ Done! Total stored matches in detailed_match_stats.json: {len(all_stats)}")

[1/2347] 1-1ad2265b-87df-422d-89a1-64085b55049f has no stats ❌
[2/2347] 1-42fb9f0a-bc5c-4012-9a07-b283fdbe1f9a has no stats ❌
[3/2347] 1-07ea9bcc-4278-4ace-b358-bea44d0d5b69 has no stats ❌
[4/2347] 1-6ac0fa5a-8485-4263-9d91-4db12aebad94 has no stats ❌
[5/2347] 1-697862c4-4cfa-4e46-96e2-1e251d0133c5 has no stats ❌
[6/2347] 1-779e93ac-615a-4f91-a939-1ad141854dba has no stats ❌
[7/2347] 1-74a39db1-537d-4e42-bc65-9616cdda7b49 has no stats ❌
[8/2347] 1-45bef5ed-c015-4b68-813f-8640e2d52931 has no stats ❌
[9/2347] 1-30726f29-4038-4e40-9695-4d6c43aacf4d has no stats ❌
[10/2347] 1-fe213b41-25c0-4715-8ec1-4083021bbd23 has no stats ❌
[11/2347] 1-1706fa4d-8768-4779-87c7-6d6e2205fd90 has no stats ❌
[12/2347] 1-165477b7-d36d-41c4-ac30-574001d973d5 has no stats ❌
[13/2347] 1-5817954d-d20d-452b-a231-bad3bef89696 has no stats ❌
[14/2347] 1-6cea49c9-c7fa-4a7b-a7cb-3d95770dbbdb has no stats ❌
[15/2347] 1-73b2dbcb-5b00-469f-b7c9-2c7010479172 has no stats ❌
[16/2347] 1-f85b2bd4-ceaa-43c3-90d6-13fa2dfb9297 

In [15]:
team_ids = set()  # set pour éviter les doublons

for match_id, match_data in all_stats.items():
    rounds = match_data.get('rounds', [])
    for round_data in rounds:
        teams = round_data.get('teams', [])
        for team in teams:
            team_id = team.get('team_id')
            if team_id:
                team_ids.add(team_id)

team_ids = list(team_ids)

### Pull details of each team involved in ESEA matches

In [19]:
team_ids_to_fetch = [team["team_id"] for team in team_list if team["team_id"]]

def fetch_team_details(team_ids, headers, team_file="team_details.json", delay=0.3, max_threads=5):
    team_path = Path(team_file)

    # Load existing data if present
    if team_path.exists():
        with team_path.open("r", encoding="utf-8") as f:
            all_teams = json.load(f)
    else:
        all_teams = {}

    # Filter team_ids that haven't been fetched yet
    to_fetch = [tid for tid in team_ids if tid not in all_teams]
    print(f"🔍 {len(to_fetch)} new teams to fetch among {len(team_ids)} total.\n")

    def fetch_single_team(team_id):
        url = f"https://open.faceit.com/data/v4/teams/{team_id}"
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 404:
                print(f"{team_id} → Team not found (404) ❌")
                return team_id, None

            response.raise_for_status()
            time.sleep(delay)
            return team_id, response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching {team_id} ❌ : {e}")
            return team_id, None

    # Threaded fetching
    if to_fetch:
        with ThreadPoolExecutor(max_workers=max_threads) as executor:
            futures = {executor.submit(fetch_single_team, tid): tid for tid in to_fetch}
            for i, future in enumerate(as_completed(futures), start=1):
                team_id, data = future.result()
                all_teams[team_id] = data
                status = "fetched ✅" if data else "failed ❌"
                print(f"[{i}/{len(to_fetch)}] {team_id} {status}")
    else:
        print("No new teams to fetch.")

    # Save updated teams
    with team_path.open("w", encoding="utf-8") as f:
        json.dump(all_teams, f, indent=2)

    print(f"\n✅ Done! Total teams stored: {len(all_teams)}")
    return all_teams

all_teams_data = fetch_team_details(team_ids_to_fetch, headers, team_file="team_details.json", delay=delay)

🔍 0 new teams to fetch among 1339 total.

No new teams to fetch.

✅ Done! Total teams stored: 1340


In [16]:
with open(r"data\Season 54\EU\Regular\team_details.json", "r", encoding="utf-8") as f:
    all_teams_data = json.load(f)

## Tables building

### dim_championship table

In [17]:
championship_details_list = []

for data in master_matches:  # juste itérer sur la liste
    if data is None:
        continue  # skip entries with no data

    row = {
        'competition_id': data.get('competition_id'),
        'competition_name': data.get('competition_name'),
        'competition_type': data.get('competition_type'),
        'region': data.get('region'),
        'started_at': data.get('finished_at'), #on utilise la valeur de finished_at comme jour de match car si le match a été dodge il n'y a pas de start_date
    }
    championship_details_list.append(row)

dim_championships = pd.DataFrame(championship_details_list)

In [18]:
dim_championships['started_at'] = pd.to_datetime(dim_championships['started_at'], unit='s', utc=True).dt.date


In [19]:
dim_championships[['organizer','season','region','division','sub_region','state','state2','state3']] = dim_championships['competition_name'].str.split(' ',expand=True)
dim_championships['state'] = dim_championships['state2'].where(
    dim_championships['state3'].isna(),
    dim_championships['state2'] + ' ' + dim_championships['state3']
)
dim_championships["format"] = np.where(
    dim_championships["state"].str.contains("Playoffs", case=False, na=False),
    "BO3",
    "BO1"
)
dim_championships.drop(columns=['state2', 'state3'], inplace=True)

In [20]:
dim_championships = dim_championships.groupby(["competition_id"]).agg({
    "organizer": "first", 
    "competition_type": "first", 
    'region':'first',
    'sub_region':'first',
    "season": "first",       
    "division": "first",               
    "state": "first",
    "started_at" :"first",
    "format":"first",
}).reset_index()

In [21]:
dim_championships

,competition_id,organizer,competition_type,region,sub_region,season,division,state,started_at,format
0,0ce757ef-9bad-4afa-bbce-58d647824222,ESEA,championship,EU,Central,S54,Open9,Regular Season,2025-08-08,BO1
1,1e94abfd-f0e8-47cc-b450-95cccaf954e5,ESEA,championship,EU,Central,S54,Open10,Playoffs A,2025-09-01,BO3
2,36a6a3d1-6024-40c5-ba26-1f33c173eb30,ESEA,championship,EU,Central,S54,Intermediate,Playoffs B,2025-09-01,BO3
3,3704ee10-0a67-4cf9-8f62-dc6f5cdfc0c1,ESEA,championship,EU,Central,S54,Entry,Playoffs A,2025-09-01,BO3
4,3cd81a7e-ed93-43ba-be08-2a1ee7ad7237,ESEA,championship,EU,Central,S54,Open1-8,Regular Season,2025-08-10,BO1
5,4410aa73-9584-4500-a7ca-1504ed6da707,ESEA,championship,EU,Central,S54,Intermediate,Regular Season,2025-08-11,BO1
6,56b2e3c1-e98d-4196-85df-af8f527ede5b,ESEA,championship,EU,Central,S54,Open10,Playoffs B,2025-09-01,BO3
7,662b2b3b-8769-4806-b207-e92b3b0b2816,ESEA,championship,EU,Central,S54,Open9,Playoffs,None,BO3
8,70a6c0b5-c870-4c31-be83-d55a811514af,ESEA,championship,EU,Central,S54,Main,Playoffs,2025-09-01,BO3
9,736302f9-3fb4-46d4-ae5c-5633e842495d,ESEA,championship,EU,Central,S54,Open1-8,Playoffs,None,BO3


### dim_teams and dim_players tables

In [22]:
team_details_list = []

for team_ids, team_data in all_teams_data.items():
    if team_data is None:
        continue  # 👈 Skip entries with no data

    row = {
        'team_id': team_data.get('team_id'),
        'team_nickname': team_data.get('nickname'),
        'team_name': team_data.get('name'),
        'team_avatar': team_data.get('avatar'),
        'team_faceit_url': team_data.get('faceit_url'),
    }
    team_details_list.append(row)

dim_teams = pd.DataFrame(team_details_list)
dim_teams = dim_teams.drop_duplicates(subset=['team_id'], keep='last')
dim_teams['team_faceit_url'] = dim_teams['team_faceit_url'].str.replace('/{lang}','')

In [23]:
#get lis of teams & players from championship request
players_details_list = []

for match_id, match_data in all_stats.items():
    for round_data in match_data['rounds']:
        for team in round_data['teams']:
            team_id = team.get("team_id")
            for player in team["players"]:
                player_id = player["player_id"]
                nickname = player["nickname"]

                row = {
                    "player_id": player_id,
                    "team_id": team_id,
                    "player_name": nickname
                }

                row.update()
                players_details_list.append(row)

players_matches = pd.DataFrame(players_details_list)
players_matches = players_matches.drop_duplicates(subset=['player_id'], keep='last')


In [24]:
#enrich with detail of players from team_detail request
players_details_list = []

for team_id, team_data in all_teams_data.items():
    if team_data is None or 'members' not in team_data:
        continue 

    for member in team_data['members']:
        row = {
            'player_id': member.get('user_id'),
            'player_country': member.get('country'),
            'player_faceit_url': member.get('faceit_url'),
            'player_avatar' : member.get('avatar')
        }
        players_details_list.append(row)

# Create DataFrame
players_team_details = pd.DataFrame(players_details_list)
players_team_details = players_team_details.drop_duplicates(subset=['player_id'], keep='last')

In [25]:
dim_players = pd.merge(players_matches, players_team_details, on='player_id',how='left')
dim_players['player_faceit_url'] = dim_players['player_faceit_url'].str.replace('/{lang}','')

In [26]:
dim_players

,player_id,team_id,player_name,player_country,player_faceit_url,player_avatar
0,b89a3885-c9be-4dae-a593-1ad03bd48784,934a2bd0-fa66-4c02-ac92-00f742e7ec55,wolterz,ru,https://www.faceit.com/players/wolterz,https://distribution.faceit-cdn.net/images/93f...
1,117cb564-1753-47c0-8a3c-7f00f2e17534,1b637421-544a-4c74-9b70-e5ee5cf541f9,-FaaT,pl,https://www.faceit.com/players/-FaaT,https://distribution.faceit-cdn.net/images/1a9...
2,d2a9afea-2411-49c2-9d7b-6283dcccef40,df862c73-6fc9-4f84-8160-22dd06293fd3,shepzor,ru,https://www.faceit.com/players/shepzor,https://distribution.faceit-cdn.net/images/62c...
3,b065a172-1af3-4ce8-9b1b-6e6a4e0297cc,45f3397b-4dae-426c-b7ec-5c743ab13373,Dante12,NaN,NaN,NaN
4,9f95b1c2-cbd7-472a-9189-e0adfea5097a,b7d71a56-69f7-409b-ac41-d1113f659153,johny01,hu,https://www.faceit.com/players/johny01,https://distribution.faceit-cdn.net/images/12a...
...,...,...,...,...,...,...
8346,2a5f083b-0ca8-478c-96db-55ec654b4992,8289f499-4465-4f1d-8b1d-73d9cc46b94b,ryb1ka,pl,https://www.faceit.com/players/rybc1a--,https://distribution.faceit-cdn.net/images/56e...
8347,353c75c6-2aeb-4a81-b182-77cabcdc7183,46e19bb2-a96b-48bf-a373-c0167b67c894,SneakyMickey,pl,https://www.faceit.com/players/SneakyMickey,https://assets.faceit-cdn.net/avatars/353c75c6...
8348,5205ec76-613e-44f2-9752-36f9102ea34b,46e19bb2-a96b-48bf-a373-c0167b67c894,Stakenowskyy,pl,https://www.faceit.com/players/Stakenowskyy,https://distribution.faceit-cdn.net/images/3eb...
8349,323a89d6-75da-4c2f-8f48-c1e40a708c48,46e19bb2-a96b-48bf-a373-c0167b67c894,DonBabiczos,pl,https://www.faceit.com/players/DonBabiczos,https://assets.faceit-cdn.net/avatars/323a89d6...


### dim_matches table

The idea is to have a composite key with the combination of match_id and match_round being unique : 

- match id being the whole match (either its a Bo1, Bo3 or Bo5)
- match_round being the id of the map played (1 if a single map is played and then increase up to 5 for Bo5)

In [57]:
dim_matches = filtered_captain_matches_df
dim_matches = dim_matches.merge(
    dim_championships[["competition_id", "format"]],
    on="competition_id",
    how="left"
)
dim_matches = dim_matches.drop_duplicates(subset=["match_id", "competition_id"])


In [58]:
dim_matches

,match_id,competition_id,match_day,winner_id,loser_id,format
0,1-f197f4c1-6c52-46f6-b66b-edd2f5830813,e61be909-1727-4de3-8201-419d89893401,2025-08-18 18:07:19,83756527-b019-449f-9660-adc29f01416f,cfdfa687-0075-4f23-8129-28eb01b37a2a,BO1
1,1-8d4a43ae-90a0-48bb-a575-c089699bca24,e61be909-1727-4de3-8201-419d89893401,2025-08-13 18:41:24,97f7d131-9ffe-4dde-a8e5-e2dcee2113ea,cfdfa687-0075-4f23-8129-28eb01b37a2a,BO1
2,1-89b9853e-781a-4ca6-bb34-5ee736e20a50,e61be909-1727-4de3-8201-419d89893401,2025-08-11 18:39:34,cfdfa687-0075-4f23-8129-28eb01b37a2a,db190109-606c-4845-a773-574dde647db3,BO1
3,1-d606f510-8000-4f18-a438-5712554f4d49,e61be909-1727-4de3-8201-419d89893401,2025-08-10 18:36:18,aeff8fa0-734c-45bb-8b9b-f18847d630f6,cfdfa687-0075-4f23-8129-28eb01b37a2a,BO1
4,1-5517b202-c958-4244-8688-d123d637e178,e61be909-1727-4de3-8201-419d89893401,2025-08-04 18:39:52,cfdfa687-0075-4f23-8129-28eb01b37a2a,dc0203c1-9604-4ad0-bdea-644b117609b3,BO1
...,...,...,...,...,...,...
9409,1-7a5d0f6e-e6ef-41bb-8504-137f483ae33f,d66d7f6f-3430-4dc3-a0e9-0fb8d9b5e68d,2025-09-01 20:16:42,0c06fb32-9c32-4a48-8967-943baf52ab6a,bye,BO3
9410,1-40b28985-c35e-4124-9f2a-30a543d730dd,4410aa73-9584-4500-a7ca-1504ed6da707,2025-08-12 18:14:10,b3e21e6e-7e76-4cba-b3af-b65b91833d6c,df862c73-6fc9-4f84-8160-22dd06293fd3,BO1
9411,1-b202e7b4-9f28-433f-8043-bcdff80d4048,e61be909-1727-4de3-8201-419d89893401,2025-07-23 18:39:19,4269aee6-d662-4d1b-80c9-495904bd65ed,7895ce74-408e-421d-b83a-05337ac700d4,BO1
9412,1-d9206de0-f03f-456e-9c56-f466e7de9f46,e61be909-1727-4de3-8201-419d89893401,2025-07-23 18:39:51,bdc870bc-c386-4488-9084-6df1ff584794,270c07be-ae43-4628-bc95-e3776f167dee,BO1


In [59]:
column_order=['match_id','competition_id','match_day','format','winner_id','loser_id']
dim_matches = dim_matches[column_order]
dim_matches = dim_matches.drop_duplicates(subset=['match_id'], keep='first')
dim_matches = dim_matches.reset_index(drop=True)

In [60]:
dim_matches

,match_id,competition_id,match_day,format,winner_id,loser_id
0,1-f197f4c1-6c52-46f6-b66b-edd2f5830813,e61be909-1727-4de3-8201-419d89893401,2025-08-18 18:07:19,BO1,83756527-b019-449f-9660-adc29f01416f,cfdfa687-0075-4f23-8129-28eb01b37a2a
1,1-8d4a43ae-90a0-48bb-a575-c089699bca24,e61be909-1727-4de3-8201-419d89893401,2025-08-13 18:41:24,BO1,97f7d131-9ffe-4dde-a8e5-e2dcee2113ea,cfdfa687-0075-4f23-8129-28eb01b37a2a
2,1-89b9853e-781a-4ca6-bb34-5ee736e20a50,e61be909-1727-4de3-8201-419d89893401,2025-08-11 18:39:34,BO1,cfdfa687-0075-4f23-8129-28eb01b37a2a,db190109-606c-4845-a773-574dde647db3
3,1-d606f510-8000-4f18-a438-5712554f4d49,e61be909-1727-4de3-8201-419d89893401,2025-08-10 18:36:18,BO1,aeff8fa0-734c-45bb-8b9b-f18847d630f6,cfdfa687-0075-4f23-8129-28eb01b37a2a
4,1-5517b202-c958-4244-8688-d123d637e178,e61be909-1727-4de3-8201-419d89893401,2025-08-04 18:39:52,BO1,cfdfa687-0075-4f23-8129-28eb01b37a2a,dc0203c1-9604-4ad0-bdea-644b117609b3
...,...,...,...,...,...,...
9409,1-7a5d0f6e-e6ef-41bb-8504-137f483ae33f,d66d7f6f-3430-4dc3-a0e9-0fb8d9b5e68d,2025-09-01 20:16:42,BO3,0c06fb32-9c32-4a48-8967-943baf52ab6a,bye
9410,1-40b28985-c35e-4124-9f2a-30a543d730dd,4410aa73-9584-4500-a7ca-1504ed6da707,2025-08-12 18:14:10,BO1,b3e21e6e-7e76-4cba-b3af-b65b91833d6c,df862c73-6fc9-4f84-8160-22dd06293fd3
9411,1-b202e7b4-9f28-433f-8043-bcdff80d4048,e61be909-1727-4de3-8201-419d89893401,2025-07-23 18:39:19,BO1,4269aee6-d662-4d1b-80c9-495904bd65ed,7895ce74-408e-421d-b83a-05337ac700d4
9412,1-d9206de0-f03f-456e-9c56-f466e7de9f46,e61be909-1727-4de3-8201-419d89893401,2025-07-23 18:39:51,BO1,bdc870bc-c386-4488-9084-6df1ff584794,270c07be-ae43-4628-bc95-e3776f167dee


In [62]:
dupes = dim_matches[dim_matches.duplicated(subset=["match_id"], keep=False)]
print(dupes)

Empty DataFrame
Columns: [match_id, competition_id, match_day, format, winner_id, loser_id]
Index: []


In [63]:
dim_matches.dtypes

match_id                  object
competition_id            object
match_day         datetime64[ns]
format                    object
winner_id                 object
loser_id                  object
dtype: object

### Dim Maps

In [86]:
dim_maps_list = []

for match_id, match_data in all_stats.items():
    for round_data in match_data['rounds']:
        total_rounds_played = round_data['round_stats'].get('Rounds')
        map = round_data["round_stats"].get("Map")
        winner_id = round_data["round_stats"].get("Winner")

        loser_id = None
        for team in round_data["teams"]:
            if team["team_id"] != winner_id:
                loser_id = team["team_id"]
                break  # only 1 loser in each map
            
        row = {
            "match_id": match_id,
            "match_round": round_data.get("match_round"),
            "map": map,
            "winner_id": winner_id,
            "loser_id": loser_id,
            "total_rounds_played": total_rounds_played
        }

        dim_maps_list.append(row)

dim_maps = pd.DataFrame(dim_maps_list)


In [87]:
dim_maps

,match_id,match_round,map,winner_id,loser_id,total_rounds_played
0,1-74860e59-e961-4a1c-868c-55ef7d3215b8,1,de_dust2,f8ac5861-2bb7-4dd1-8378-afa7a9257920,30191a9c-ceb2-4da0-a146-ec375782ac45,12
1,1-14e4f8cb-ca01-47f8-aee1-19136f852ea4,1,de_nuke,36ef04e5-fdbe-43bc-b204-3c6d771dca4e,6d406cd9-c6a2-4c0d-aa9d-45dfa5812460,23
2,1-0b1fb845-4c11-42af-8026-743757ba84ea,1,de_anubis,aa598c46-986e-4f63-b06e-b01099a05c37,eb3fe6c6-def5-4ee6-876d-8014f3a038c9,30
3,1-2f8d7a69-47a4-4f2f-b3e1-b4003e532d0e,1,de_dust2,86ff2e09-b46b-42c6-b0ca-d3dd9de5e4da,babdf840-16de-48ee-aa64-bc8dec967fbc,19
4,1-bb0019cd-142e-4f6a-a87f-b3d92889ba69,1,de_inferno,2f38c7a9-53ea-41ad-8920-e4c7b9c7cd7c,b95adac1-945a-4e24-b8b7-a4266bde069c,15
...,...,...,...,...,...,...
7429,1-9b11181c-179c-4d53-b058-c0f8d911770d,2,de_overpass,669ee56b-b606-4f57-a316-066a17926f59,6bb0008b-e18f-4723-a265-cc6f8538efe4,30
7430,1-2353f166-8087-4e43-9f0d-8ad8c7b2a320,1,de_overpass,93bd263e-e822-4af3-9cb9-4c1a58b3eb34,d1210030-4766-40b9-84cf-dd64171ca456,21
7431,1-2353f166-8087-4e43-9f0d-8ad8c7b2a320,2,de_dust2,d1210030-4766-40b9-84cf-dd64171ca456,93bd263e-e822-4af3-9cb9-4c1a58b3eb34,18
7432,1-2353f166-8087-4e43-9f0d-8ad8c7b2a320,3,de_nuke,d1210030-4766-40b9-84cf-dd64171ca456,93bd263e-e822-4af3-9cb9-4c1a58b3eb34,24


In [88]:
dim_maps = dim_maps.drop_duplicates(subset=['match_id', 'match_round'], keep='last')

### fact_players_stats table 

In [122]:
player_stats_list = []

for match_id, match_data in all_stats.items():
    for round_data in match_data['rounds']:
        map_name = round_data['round_stats'].get('Map')
        winner_id = round_data['round_stats'].get('Winner')

        for team in round_data['teams']:
            team_id = team.get("team_id")
            team_name = team["team_stats"].get("Team")
            team_win = team["team_stats"].get("Team Win")

            for player in team["players"]:
                player_id = player["player_id"]
                nickname = player["nickname"]
                stats = player["player_stats"]

                row = {
                    "match_id": match_id,
                    "match_round": round_data.get("match_round"),
                    "player_id": player_id,
                    "team_id": team_id,
                }

                row.update(stats)
                player_stats_list.append(row)

fact_players_stats = pd.DataFrame(player_stats_list)


In [123]:
#enrich with competition_id and match_day
fact_players_stats = pd.merge(fact_players_stats, dim_matches[['match_id','competition_id','match_day']], on=['match_id'])

In [124]:
#correction d'un cas ou les données d'une game sont en doublons, on garde les lignes qui ont les stats les plus importantes, exemple :https://www.faceit.com/fr/cs2/room/1-e4410b46-a6ac-4037-ad35-d7f91f8bf55f/scoreboard
players_stats_sorted = fact_players_stats.sort_values(['match_id', 'player_id', 'Kills'], ascending=[True, True, False])
fact_players_stats = players_stats_sorted.drop_duplicates(subset=['match_id', 'player_id'], keep='first')
#correction d'un bug ou un joueur sur la fiche d'un match mais n'ayant pas joué se retrouve avec des stats a 0 (nouvelle logique de récupération des matchs via captains)
fact_players_stats = fact_players_stats[fact_players_stats['Damage']!= 0]

In [125]:
fact_players_stats = fact_players_stats.drop(columns={'Result','Headshots %','K/R Ratio','K/D Ratio','Match Entry Rate','Match 1v1 Win Rate','Flash Success Rate per Match','Sniper Kill Rate per Round','Utility Usage per Round','Flashes per Round in a Match','Sniper Kill Rate per Match','Utility Damage per Round in a Match','Utility Damage Success Rate per Match','Utility Successes','Enemies Flashed per Round in a Match','Utility Success Rate per Match','Match Entry Success Rate','Match 1v2 Win Rate','Utility Enemies'})

In [126]:
fact_players_stats = fact_players_stats.rename(columns={
    'Zeus Kills':'zeus_kills',
    'Utility Count':'utility_count',
    'Double Kills':'double_kills',
    'Flash Successes':'flash_successes',
    'Quadro Kills':'quadro_kills',
    'Entry Count':'entry_count',
    'First Kills':'first_kills',
    'Flash Count':'flash_count',
    'Sniper Kills':'sniper_kills',
    'Damage':'damage',
    'Utility Damage':'utility_damage',
    'Assists':'assists',
    '1v1Count':'count_1v1',
    'Enemies Flashed':'enemies_flashed',
    'Clutch Kills':'clutch_kills',
    'Penta Kills':'ace',
    'MVPs':'mvps',
    'Deaths':'deaths',
    'Entry Wins':'entry_wins',
    'Kills':'kills',
    '1v2Wins':'wins_1v2',
    '1v1Wins':'wins_1v1',
    'Pistol Kills':'pistol_kills',
    'Knife Kills':'knife_kills',
    'ADR':'adr',
    '1v2Count':'count_1v2',
    'Triple Kills':'triple_kills',
    'Headshots':'headshots',
})

In [127]:
column_order=['match_id', 'match_round','player_id', 'competition_id','team_id','zeus_kills',
'utility_count',
'double_kills',
'flash_successes',
'quadro_kills',
'entry_count',
'first_kills',
'flash_count',
'sniper_kills',
'damage',
'utility_damage',
'assists',
'count_1v1',
'enemies_flashed',
'clutch_kills',
'ace',
'mvps',
'deaths',
'entry_wins',
'kills',
'wins_1v2',
'wins_1v1',
'pistol_kills',
'knife_kills',
'adr',
'count_1v2',
'triple_kills',
'headshots', 
'match_day']
fact_players_stats = fact_players_stats[column_order]

In [128]:
fact_players_stats.columns

Index(['match_id', 'match_round', 'player_id', 'competition_id', 'team_id',
       'zeus_kills', 'utility_count', 'double_kills', 'flash_successes',
       'quadro_kills', 'entry_count', 'first_kills', 'flash_count',
       'sniper_kills', 'damage', 'utility_damage', 'assists', 'count_1v1',
       'enemies_flashed', 'clutch_kills', 'ace', 'mvps', 'deaths',
       'entry_wins', 'kills', 'wins_1v2', 'wins_1v1', 'pistol_kills',
       'knife_kills', 'adr', 'count_1v2', 'triple_kills', 'headshots',
       'match_day'],
      dtype='object')

In [129]:
exclude_cols = ['match_id','match_round','player_id','competition_id','team_id','match_day']

for col in fact_players_stats.columns:
    if col not in exclude_cols:
        fact_players_stats[col] = pd.to_numeric(fact_players_stats[col], errors="ignore")


C:\Users\Armel\AppData\Local\Temp\ipykernel_28020\4106138510.py:5: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  fact_players_stats[col] = pd.to_numeric(fact_players_stats[col], errors="ignore")


In [130]:
fact_players_stats.dtypes

match_id                   object
match_round                object
player_id                  object
competition_id             object
team_id                    object
zeus_kills                  int64
utility_count               int64
double_kills                int64
flash_successes             int64
quadro_kills                int64
entry_count                 int64
first_kills                 int64
flash_count                 int64
sniper_kills                int64
damage                      int64
utility_damage              int64
assists                     int64
count_1v1                   int64
enemies_flashed             int64
clutch_kills                int64
ace                         int64
mvps                        int64
deaths                      int64
entry_wins                  int64
kills                       int64
wins_1v2                    int64
wins_1v1                    int64
pistol_kills                int64
knife_kills                 int64
adr           

In [131]:
dupes = fact_players_stats[fact_players_stats.duplicated(subset=["match_id", "match_round","player_id"], keep=False)]
print(dupes)

Empty DataFrame
Columns: [match_id, match_round, player_id, competition_id, team_id, zeus_kills, utility_count, double_kills, flash_successes, quadro_kills, entry_count, first_kills, flash_count, sniper_kills, damage, utility_damage, assists, count_1v1, enemies_flashed, clutch_kills, ace, mvps, deaths, entry_wins, kills, wins_1v2, wins_1v1, pistol_kills, knife_kills, adr, count_1v2, triple_kills, headshots, match_day]
Index: []

[0 rows x 34 columns]


### fact_teams_stats table 

In [79]:
team_stats_list = []

for match_id, match_data in all_stats.items():
    for round_data in match_data['rounds']:
        total_rounds_played = round_data['round_stats'].get('Rounds')
        map = round_data["round_stats"].get("Map")
        
        for team in round_data['teams']:
            team_id = team.get("team_id")
            team_name = team["team_stats"].get("Team")
            team_win = team["team_stats"].get("Team Win")
            
            row = {
                "match_id": match_id,
                "match_round": round_data.get("match_round"),
                "map": map,
                "team_id": team_id,
                "team_win": team_win,
                "total_rounds_played": total_rounds_played
            }

            row.update(team["team_stats"])
            team_stats_list.append(row)

fact_teams_stats = pd.DataFrame(team_stats_list)


In [80]:
fact_teams_stats.head(2)

,match_id,match_round,map,team_id,team_win,total_rounds_played,First Half Score,Team,Team Win,Final Score,Team Headshots,Second Half Score,Overtime score
0,1-74860e59-e961-4a1c-868c-55ef7d3215b8,1,de_dust2,30191a9c-ceb2-4da0-a146-ec375782ac45,0,12,1,officers,0,1,3.2,0,0
1,1-74860e59-e961-4a1c-868c-55ef7d3215b8,1,de_dust2,f8ac5861-2bb7-4dd1-8378-afa7a9257920,1,12,11,The Toxics,1,11,4.6,0,0


In [81]:
# Enrich with competition_id and match_day - only merge on match_id because comp_id and match_day doesnt differ depending on match_round
fact_teams_stats = pd.merge(fact_teams_stats, dim_matches[['match_id','competition_id','match_day']], on='match_id')

In [82]:
fact_teams_stats

,match_id,match_round,map,team_id,team_win,total_rounds_played,First Half Score,Team,Team Win,Final Score,Team Headshots,Second Half Score,Overtime score,competition_id,match_day
0,1-74860e59-e961-4a1c-868c-55ef7d3215b8,1,de_dust2,30191a9c-ceb2-4da0-a146-ec375782ac45,0,12,1,officers,0,1,3.2,0,0,3cd81a7e-ed93-43ba-be08-2a1ee7ad7237,2025-07-30 18:40:44
1,1-74860e59-e961-4a1c-868c-55ef7d3215b8,1,de_dust2,f8ac5861-2bb7-4dd1-8378-afa7a9257920,1,12,11,The Toxics,1,11,4.6,0,0,3cd81a7e-ed93-43ba-be08-2a1ee7ad7237,2025-07-30 18:40:44
2,1-14e4f8cb-ca01-47f8-aee1-19136f852ea4,1,de_nuke,6d406cd9-c6a2-4c0d-aa9d-45dfa5812460,0,23,7,5PellMans,0,10,8.6,3,0,4410aa73-9584-4500-a7ca-1504ed6da707,2025-08-19 18:13:11
3,1-14e4f8cb-ca01-47f8-aee1-19136f852ea4,1,de_nuke,36ef04e5-fdbe-43bc-b204-3c6d771dca4e,1,23,5,WHYNOT,1,13,8,8,0,4410aa73-9584-4500-a7ca-1504ed6da707,2025-08-19 18:13:11
4,1-0b1fb845-4c11-42af-8026-743757ba84ea,1,de_anubis,aa598c46-986e-4f63-b06e-b01099a05c37,1,30,6,Tiim,1,16,9,6,4,f62c4c75-f620-40a2-bef8-0be2cb85d2d3,2025-07-16 18:37:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14862,1-2353f166-8087-4e43-9f0d-8ad8c7b2a320,2,de_dust2,93bd263e-e822-4af3-9cb9-4c1a58b3eb34,0,18,4,ONLY POSITIVE,0,5,4.6,1,0,36a6a3d1-6024-40c5-ba26-1f33c173eb30,2025-09-02 19:42:32
14863,1-2353f166-8087-4e43-9f0d-8ad8c7b2a320,3,de_nuke,d1210030-4766-40b9-84cf-dd64171ca456,1,24,3,Erciyes,1,13,9.8,10,0,36a6a3d1-6024-40c5-ba26-1f33c173eb30,2025-09-02 19:42:32
14864,1-2353f166-8087-4e43-9f0d-8ad8c7b2a320,3,de_nuke,93bd263e-e822-4af3-9cb9-4c1a58b3eb34,0,24,9,ONLY POSITIVE,0,11,8.2,2,0,36a6a3d1-6024-40c5-ba26-1f33c173eb30,2025-09-02 19:42:32
14865,1-0623043b-fd49-45bf-ae17-092e3e291c3b,1,de_mirage,8289f499-4465-4f1d-8b1d-73d9cc46b94b,1,15,10,Stoned Goats,1,13,6.4,3,0,56b2e3c1-e98d-4196-85df-af8f527ede5b,2025-09-02 18:26:03


In [83]:
fact_teams_stats = fact_teams_stats.drop(columns={'Team','Team Win','Team Headshots'})
fact_teams_stats = fact_teams_stats.rename(columns={'Final Score':'final_score','Overtime score':'overtime_score','First Half Score':'first_half_score','Second Half Score':'second_half_score'})

In [84]:
fact_teams_stats.head(2)

,match_id,match_round,map,team_id,team_win,total_rounds_played,first_half_score,final_score,second_half_score,overtime_score,competition_id,match_day
0,1-74860e59-e961-4a1c-868c-55ef7d3215b8,1,de_dust2,30191a9c-ceb2-4da0-a146-ec375782ac45,0,12,1,1,0,0,3cd81a7e-ed93-43ba-be08-2a1ee7ad7237,2025-07-30 18:40:44
1,1-74860e59-e961-4a1c-868c-55ef7d3215b8,1,de_dust2,f8ac5861-2bb7-4dd1-8378-afa7a9257920,1,12,11,11,0,0,3cd81a7e-ed93-43ba-be08-2a1ee7ad7237,2025-07-30 18:40:44


In [85]:
#correction d'un cas ou les données d'une game sont en doublons, on garde les lignes qui ont les stats les plus importantes, exemple :https://www.faceit.com/fr/cs2/room/1-e4410b46-a6ac-4037-ad35-d7f91f8bf55f/scoreboard
team_stats_sorted = fact_teams_stats.sort_values(['match_id', 'team_id', 'final_score'], ascending=[True, True, False])
fact_teams_stats = team_stats_sorted.drop_duplicates(subset=['match_id', 'team_id'], keep='first')

column_order=['match_id','match_round','team_id','competition_id','team_win','total_rounds_played','first_half_score','second_half_score','overtime_score','final_score']
fact_teams_stats = fact_teams_stats[column_order]

In [86]:
fact_teams_stats.columns

Index(['match_id', 'match_round', 'team_id', 'competition_id', 'team_win',
       'total_rounds_played', 'first_half_score', 'second_half_score',
       'overtime_score', 'final_score'],
      dtype='object')

In [87]:
fact_teams_stats.dtypes

match_id               object
match_round            object
team_id                object
competition_id         object
team_win               object
total_rounds_played    object
first_half_score       object
second_half_score      object
overtime_score         object
final_score            object
dtype: object

In [72]:
len(fact_teams_stats)

14439

## Summary of tables & columns

In [57]:
dim_championships.columns

Index(['competition_id', 'organizer', 'competition_type', 'region',
       'sub_region', 'season', 'division', 'state', 'started_at'],
      dtype='object')

In [58]:
dim_teams.columns

Index(['team_id', 'team_nickname', 'team_name', 'team_avatar',
       'team_faceit_url'],
      dtype='object')

In [59]:
dim_players.columns

Index(['player_id', 'team_id', 'player_name', 'player_country',
       'player_faceit_url', 'player_avatar'],
      dtype='object')

In [60]:
dim_matches.columns

Index(['match_id', 'match_round', 'competition_id', 'match_day', 'format',
       'map', 'total_rounds_played', 'winner_id', 'loser_id'],
      dtype='object')

In [61]:
fact_players_stats.columns

Index(['match_id', 'match_round', 'player_id', 'competition_id', 'team_id',
       'zeus_kills', 'utility_count', 'double_kills', 'flash_successes',
       'quadro_kills', 'entry_count', 'first_kills', 'flash_count',
       'sniper_kills', 'damage', 'utility_damage', 'assists', 'count_1v1',
       'enemies_flashed', 'clutch_kills', 'ace', 'mvps', 'deaths',
       'entry_wins', 'kills', 'wins_1v2', 'wins_1v1', 'pistol_kills',
       'knife_kills', 'adr', 'count_1v2', 'triple_kills', 'headshots',
       'match_day'],
      dtype='object')

In [62]:
fact_teams_stats.columns

Index(['match_id', 'match_round', 'team_id', 'competition_id', 'team_win',
       'total_rounds_played', 'first_half_score', 'second_half_score',
       'overtime_score', 'final_score'],
      dtype='object')

In [57]:
# Find IDs in fact that are missing in dim
missing_in_dim = fact_players_stats.loc[~fact_players_stats["player_id"].isin(dim_players["player_id"]), "player_id"].unique()

# Find IDs in dim that are missing in fact
missing_in_fact = dim_players.loc[~dim_players["player_id"].isin(fact_players_stats["player_id"]), "player_id"].unique()

print(f"IDs in fact but not in dim: {missing_in_dim}")
print(f"IDs in dim but not in fact: {missing_in_fact}")

# Optional: get the actual fact rows with missing IDs
fact_missing_rows = fact_players_stats[fact_players_stats["player_id"].isin(missing_in_dim)]


IDs in fact but not in dim: []
IDs in dim but not in fact: []


## Data ingestion on PostgreSQL DB

In [79]:
engine = create_engine(
    f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
)

In [93]:
primary_keys_map = {
    "dim_championships": ["competition_id"],
    "dim_teams": ["team_id"],
    "dim_players": ["player_id"],
    "dim_matches": ["match_id"],
    "dim_maps":['match_id','match_round'],
    "fact_teams_stats": ["match_id", "match_round", "team_id"],
    "fact_players_stats": ["match_id", "match_round", "player_id"]
}

def insert_new_records(df, table_name, pk_cols, chunk_size=5000):
    try:
        pk_cols_str = ', '.join(pk_cols)
        query = f"SELECT {pk_cols_str} FROM {table_name}"
        
        existing_pks_df = pd.read_sql(query, engine)

        # Normalize keys to string to avoid datatype mismatches
        for col in pk_cols:
            df[col] = df[col].astype(str)
            existing_pks_df[col] = existing_pks_df[col].astype(str)

        if len(pk_cols) == 1:
            col = pk_cols[0]
            mask = ~df[col].isin(existing_pks_df[col])
        else:
            df_keys = df[pk_cols].apply(tuple, axis=1)
            existing_keys = existing_pks_df.apply(tuple, axis=1)
            mask = ~df_keys.isin(existing_keys)
        
        new_records_df = df[mask]
        
        if not new_records_df.empty:
            new_records_df.to_sql(
                table_name,
                con=engine,
                if_exists='append',
                index=False,
                method='multi',
                chunksize=chunk_size  # <-- added chunking
            )
            print(f"Inserted {len(new_records_df)} new records into {table_name}")
        else:
            print(f"No new records to insert for {table_name}")
    except SQLAlchemyError as e:
        print(f"Database error while inserting into {table_name}: {e}")
    except Exception as e:
        print(f"Unexpected error while inserting into {table_name}: {e}")

def main():
    try:
        # insert_new_records(dim_championships, "dim_championships", primary_keys_map["dim_championships"])
        # insert_new_records(dim_teams, "dim_teams", primary_keys_map["dim_teams"])
        # insert_new_records(dim_players, "dim_players", primary_keys_map["dim_players"])
        # insert_new_records(dim_matches, "dim_matches", primary_keys_map["dim_matches"])
        insert_new_records(dim_maps, "dim_maps", primary_keys_map["dim_maps"])
        # insert_new_records(fact_teams_stats, "fact_teams_stats", primary_keys_map["fact_teams_stats"])
        # insert_new_records(fact_players_stats, "fact_players_stats", primary_keys_map["fact_players_stats"])
    except Exception as e:
        print(f"Error in main loading process: {e}")

if __name__ == "__main__":
    main()


C:\Users\Armel\AppData\Local\Temp\ipykernel_34392\200828122.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(str)
C:\Users\Armel\AppData\Local\Temp\ipykernel_34392\200828122.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(str)


Inserted 7431 new records into dim_maps
